# Import the MIMIC data

In [1]:
import gzip
import pandas as pd
import glob

dataset_dictionary = {}

for file_path in glob.glob('.\\Data\\MIMIC Files\*'):
    file_name = file_path.split('\\')[3].split('.')[0]
    with gzip.open(file_path, mode='r') as file:
        dataset_dictionary[file_name] = pd.read_csv(file)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Explore Data & Clean the Data

In [100]:
# ------------------------------------------------------
# Explore 
# ------------------------------------------------------


# dataset_dictionary['NOTEEVENTS'].shape # (2083180, 11)
# dataset_dictionary['NOTEEVENTS'].info()

## Checking for distinct categories
# set(dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] ) 

# ------------------------------------------------------
# Join the tables - Draft
# ------------------------------------------------------

# 1 - Check for primary keys and distinct columns to join on -----

# 1a - Concatenate row_id and subject_id and compare lengths to unique values -----
# Answer: Join on the subject and HADM ID's and subset to one category type (e.g. discharge summary)

# # Join columns together to find a unique column
# note_events = dataset_dictionary['NOTEEVENTS']
# note_events['distinctcheck'] = note_events['HADM_ID'].astype(str) + ' | ' + note_events['SUBJECT_ID'].astype(str)
# print(note_events['distinctcheck'].nunique(), note_events.shape[0])

# # Filter to an example
# x = dataset_dictionary['NOTEEVENTS'][(dataset_dictionary['NOTEEVENTS'].loc[:,'SUBJECT_ID'] == 22532) \
#                                  & (dataset_dictionary['NOTEEVENTS'].loc[:,'HADM_ID'] == 167853) \
#                                 & (dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary')]

# # It wasn't quite unique, but if I use this join, it should combine all the notes
# l = x.groupby(['SUBJECT_ID', 'HADM_ID'])['TEXT'].agg(sum)
# display(l.values)

## 1b - Check the CPT table -----
# """The CPT tables has duplicate rows, but that's ok since text will be associated for every different HADM_ID. I do
# think that I need to create a distinct dataset for each HADM_ID though, just reduce it to just each unique HADM_ID
# and the CPT's"""

# dataset_dictionary['CPTEVENTS'].loc[:, 'HADM_ID'] == 129545
# dataset_dictionary['CPTEVENTS'][dataset_dictionary['CPTEVENTS'].loc[:, 'HADM_ID'] == 129545]

# 2 Create dataset to join together -----

# # 2a Create note_events table -----
# # Combine text for each subject and encounter
# note_events_base = dataset_dictionary['NOTEEVENTS'][dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary']
# note_events = note_events_base.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['TEXT'].agg(sum)

# # Check for dups
# note_events.loc[:, ['SUBJECT_ID', 'HADM_ID']].duplicated().any() # no dups

# 2b Create CPT table -----

# cpt_events_base = dataset_dictionary['CPTEVENTS'].loc[:, ['SUBJECT_ID','HADM_ID', 'CPT_CD']]

# # print(cpt_events_base.shape, cpt_events_base.drop_duplicates().shape) # Cut the number of records pretty much in half
# cpt_events = cpt_events_base.drop_duplicates()

# 3 Join the datasets -----

# note_events

# ------------------------------------------------------
# Preprocessing
# ------------------------------------------------------

# 1 - Remove all data not pertinent to an ICD diagnosis ---------------
# dataset_dictionary['NOTEEVENTS']



,SUBJECT_ID,HADM_ID,CPT_CD
0,11743,129545,99232
4,6185,183725,99223
5,3663,126380,99223
6,3663,126380,99233
7,3663,126380,99232
...,...,...,...
573099,78870,164649,94002
573106,78873,168759,94003
573107,78876,163404,94003
573142,78879,136071,94003


# Join the tables

In [103]:
# Create dataset to join together -----

# Create note_events table -----

# Combine text for each subject and encounter
note_events_base = dataset_dictionary['NOTEEVENTS'][dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary']
note_events = note_events_base.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['TEXT'].agg(sum)

# Create CPT table -----

cpt_events_base = dataset_dictionary['CPTEVENTS'].loc[:, ['SUBJECT_ID','HADM_ID', 'CPT_CD']]
cpt_events = cpt_events_base.drop_duplicates()

# Join the datasets -----

note_cpt = note_events.merge(cpt_events, on = ['SUBJECT_ID','HADM_ID'])
# print(note_cpt.shape, note_events.shape, cpt_events.shape) # (223,150, 4) (52,726, 3) (227,510, 3)

(223150, 4) (52726, 3) (227510, 3)


# Tokenize Data

In [111]:
# Import Packages -----

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data -----

X_train, X_test, y_train, y_test = train_test_split(note_cpt['TEXT'], note_cpt['CPT_CD'], test_size = .33, random_state = 42)

# Tokenize the data -----

# Import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df = .7)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)

# print(tfidf_vectorizer.get_feature_names()[:10])

['00', '000', '0000', '0000am', '0000hrs', '0000s', '0000units', '0001', '0003', '0004']


# Run Naive Bayes

In [118]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB()

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train.astype(str))
pred = nb_classifier.predict(tfidf_test)

TypeError: '<' not supported between instances of 'int' and 'str'

In [127]:
# metrics.accuracy_score(y_test.astype(str), pred)

"""
V1 NLP Model Accuracy: 0.117
Wow, I've got a long way to go to improve accuracy
"""

# Confusion matrix 
# confusion_mtrx = metrics.confusion_matrix(y_test.astype(str), pred) # 1380, 1380

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
# NLP V2

"""
Here are the changes I am making to improve the score:
* I will be using the max_df arg of the TfidfVectorizer function

"""

In [ ]:
X_train

# Explore the training data